This notebook explores AI agents, specifically using the LangChain framework. Think of an agent as a smart system that uses a Large Language Model (LLM) not just to answer questions, but to reason and decide on a sequence of actions. To do this, we give the agent access to tools—like a search engine, a calculator, or even custom Python code. The agent's job is to figure out which tools to use, and in what order, to accomplish a task it couldn't solve on its own.

### Getting Started

In [ ]:
!pip install -q langchain-google-genai google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
import requests

# Get the API key from Colab secrets
API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure the genai library with the API key
genai.configure(api_key=API_KEY)

### Perform Authentiation

In [ ]:
from google.colab import auth
auth.authenticate_user()

### Test your API KEY

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=API_KEY)
try:
    response = llm.invoke("What is LLM?")
    print(response)
except Exception as e:
    print(f"An error occurred: {e}")


### Python function as a tool

In [ ]:
# Currency converter function
def convert_currency(input_text: str) -> str:
    """
    Input format: '100 USD to INR'
    """
    try:
        parts = input_text.strip().upper().split()
        amount = float(parts[0])
        from_currency = parts[1]
        to_currency = parts[3]

        # Call exchange rate API
        url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
        response = requests.get(url)
        data = response.json()

        rate = data["rates"].get(to_currency)
        if rate is None:
            return f"Conversion rate not found for {to_currency}."

        converted = amount * rate
        return f"{amount} {from_currency} = {converted:.2f} {to_currency}"

    except Exception as e:
        return f"Error: {str(e)}. Format should be like '100 USD to INR'"

In [ ]:
from langchain.agents import Tool

currency_tool = Tool(
    name="CurrencyConverter",
    func=convert_currency,
    description="Use this tool to convert currency like '100 USD to INR'."
)

### Define your fist Agent

In [ ]:
from langchain.agents import initialize_agent


agent = initialize_agent(
    tools=[currency_tool],
    llm=llm,
    verbose=True
)

### Test your Agent

In [ ]:
response = agent.run("Convert 250 EUR to JPY")
print(response)

### Adding another tool

In [ ]:
# One more tool
def multiply(text: str) -> str:
    a, b = map(int, text.split())
    return str(a * b)

multiply_tool = Tool(
    name="Multiply",
    func=multiply,
    description="Multiplies two numbers provided as a space-separated string."
)


### Redefine Agent

In [ ]:
agent = initialize_agent(
    tools=[currency_tool,multiply_tool],
    llm=llm,
    verbose=True
)

### Test the Agent with Multiple Tools

In [ ]:
response = agent.run("what is 5 times 4? ")
print(response)

In [ ]:
response = agent.run("100 indian rupees equals to how many US ?")
print(response)